In [1]:
import sys
import os
import tensorflow as tf
from keras.models import load_model
from tensorflow.keras import layers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from skimage.io import imread
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import pandas as pd

In [2]:
K.clear_session()

In [3]:
data_entrenamiento = './train'
data_validacion = './test'

In [4]:
#Parametros red neuronal

epocas = 2
altura, longitud = 100, 100
batch_size = 32
pasos = 200
pasos_validacion = 50
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 29
lr = 0.0005

In [5]:
images_df = pd.read_csv(r'.\\file_target.csv')

images_df = images_df.rename(columns={'archivo': 'filenames',
                       'target': 'category'})

images_df = images_df.sort_values('category')

images_df['category'].unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'del', 'nothing', 'space'], dtype=object)

In [6]:
entrenamiento, validacion = train_test_split(images_df, test_size = 0.19)

entrenamiento = entrenamiento.reset_index(drop=True)

entrenamiento = entrenamiento.sort_values('category')

validacion = validacion.reset_index(drop=True)

validacion = validacion.sort_values('category')

In [7]:
#Pre-procesamiento de las imágenes
#Primero creamos un generador que dice como vamos a preprocesar la info

entrenamiento_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                          shear_range = 0.3,
                                          zoom_range = 0.3,
                                          horizontal_flip = True)

validacion_datagen = ImageDataGenerator(rescale = 1.0/255.0)

imagen_entrenamiento = entrenamiento_datagen.flow_from_dataframe(entrenamiento,
                                                                 data_entrenamiento,
                                                                 target_size=(altura, longitud),
                                                                 x_col='filenames',
                                                                y_col='category',
                                                                batch_size=batch_size,
                                                                class_mode='categorical')

imagen_validacion = validacion_datagen.flow_from_dataframe(validacion,
                                                           data_entrenamiento,
                                                          target_size=(altura, longitud),
                                                           x_col='filenames',
                                                            y_col='category',
                                                          batch_size=batch_size,
                                                          class_mode='categorical')

Found 7290 validated image filenames belonging to 29 classes.
Found 1710 validated image filenames belonging to 29 classes.


In [8]:
from tensorflow.keras.applications.vgg16 import VGG16

base_vgg16 = VGG16(input_shape=(altura, longitud, 3),
             include_top=False,
             )

"""for layer in base_vgg16.layers:
    layer.trainable = False"""

x = layers.Flatten()(base_vgg16.output)

x = layers.Dense(29, activation='relu')(x)

x = layers.Dense(29, activation='relu')(x)

x = layers.Dropout(0.5)(x)

x = layers.Dense(29, activation='sigmoid')(x)

model = tf.keras.models.Model(base_vgg16.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [9]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [ ]:
model.fit(imagen_entrenamiento, steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion, validation_steps=pasos_validacion)

dir = './modelo_vgg16/'

if not os.path.exists(dir):
    os.mkdir(dir)
model.save('./modelo_vgg16/modelo_vgg16.h5')
model.save_weights('./modelo_vgg16/pesos_vgg16.h5')